In [21]:
import requests
from requests.auth import HTTPBasicAuth
from urllib.parse import * 
import pandas as pd
import tldextract
import re
import cryptography
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.x509.oid import NameOID
import time
import statistics
import numpy as np

In [2]:
#Prints out all fields in the cert object
def displayCert(cert):
    print("Subject: " + x509.Name.rfc4514_string(cert.subject))
    print("Issuer: " + x509.Name.rfc4514_string(cert.issuer))
    print("Valid Start: " + str(cert.not_valid_before))
    print("Valid End: " + str(cert.not_valid_after))
    print("Extensions: \n")
    for ext in cert.extensions:
        print(ext)

#Get the Subject field from the cert and outputs it in a dictionary
def getCertSubj(cert):
    mydct = {}
    s = x509.Name.rfc4514_string(cert.subject)
    if len(s) == 0:
        return {}
    s = s.replace('\,', '')
    s = s.split(',')
    for f in s:
        f = f.split('=')
        mydct[f[0]] = f[1]
    return mydct

#Converts the raw data cert into a cert object
def rawToCert(rawData):
    #convert raw data into PEM byte format
    rawData = '\n'.join(rawData[i:i+64] for i in range(0, len(rawData), 64))
    pemData = '-----BEGIN CERTIFICATE-----\n' + rawData + '\n-----END CERTIFICATE-----\n'
    pemData = bytes(pemData, 'utf-8')
    #convert to cert object
    cert = x509.load_pem_x509_certificate(pemData, default_backend())
    return cert

In [24]:
#Import the list of phishing urls
df = pd.read_csv (r'benign_datasetV2.csv')
cloudflare = ['Cloudflare Inc ECC CA-3', 'Cloudflare Inc RSA CA-2']
df = df.loc[df['Isssuer CN'].isin(cloudflare)]
phish = df['Domain'].tolist()
domainList = phish

In [ ]:
#Import the list of phishing urls
df = pd.read_csv (r'online-valid.csv')
phish = df['url'].tolist()
#test using 10 phishing url for now
phishSites = phish[:10]
#extract just the domain from the url
domainList = ["{}.{}".format(tldextract.extract(p).domain, tldextract.extract(p).suffix) for p in phishSites]

In [ ]:
completed = set()

with open('phishingList.txt', 'r') as f, open('Output.txt','w') as o:
    for line in f:
        parse_url = urlparse(line).netloc #Gives URL in a 6 category format. This allows us to focus just on the netloc category
        
        if parse_url.startswith('www'):
            parse_url = parse_url.replace('www.','') #Removing 'www.' to allow proper formatting for Censys
        
        if parse_url not in completed: #Removing duplicates
            o.write(parse_url+'\n')
            completed.add(parse_url)
domainList = list(completed)
domainList = domainList[1100:]

In [25]:
#go through each domain to get certificate features
certList = []
timeperiod = []
for domain in domainList:
    startCode = time.time()
    print(str(domainList.index(domain))+ ': ' + domain)
    time.sleep(4)
    baseURL = 'https://api.certspotter.com/v1/issuances?domain='
    endURL = '&include_subdomains=true&match_wildcards=true&expand=dns_names&expand=issuer&expand=cert&expand=issuer.operator'
    url = baseURL + domain + endURL
    try:
        r = requests.get(url, auth=HTTPBasicAuth('39209_OPvjskKtppF579vX8E56', ''))
    except:
        continue
    json = r.json()
    r.close()
    #sort certificates by latest expiry date first
    #print(json)
    try:
        json = sorted(json, key=lambda k: k['not_after'], reverse=True)
    except:
        continue
    #print(len(json))
    for entry in json:
        #add subject column
        rawData = entry['cert']['data']
        cert = rawToCert(rawData)
        issuerCN = cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)
        if issuerCN != []:
            if issuerCN[0].value == 'Cloudflare Inc ECC CA-3':
    #         if entry['cert']['type'] != 'precert':

                #add new column called url for domain name
                entry['url'] = domain

                #add subject column
                rawData = entry['cert']['data']
                cert = rawToCert(rawData)
                certSubj = getCertSubj(cert)
                entry['subject'] = certSubj

                #split the issuer name into separate columns for CN, O, C etc.
                s = entry['issuer']['name']
                pattern = "\"(.*?)\""

                substrings = re.findall(pattern, s)
                newSubstrings = [x.replace(',', '') for x in substrings ] 

                for i in range(len(substrings)):
                    s = s.replace(substrings[i], newSubstrings[i])

                s = s.split(',')
                for f in s:
                    f = f.split('=')
                    entry['issuer'][f[0]] = f[1]
                certList.append(entry)
                endCode = time.time() - startCode
                timeperiod.append(endCode)
                print(endCode)
                break

0: medium.com
4.4426188468933105
1: creativecommons.org
6.586970567703247
2: www.who.int
4.28961181640625
3: tinyurl.com
4.508349418640137
4: www.cloudflare.com
4.526231050491333
5: sciencedirect.com
4.738271951675415
6: www.nginx.com
4.419347286224365
7: hugedomains.com
4.738388538360596
8: www.arnebrachhold.de
4.5346763134002686
9: themeforest.net
4.787596702575684
10: www.addtoany.com
4.446352243423462
11: www.ietf.org
4.383225917816162
12: digg.com
5.610362529754639
13: www.webmd.com
5.669249057769775
14: www.shopify.com
5.308945894241333
15: allaboutcookies.org
4.34523606300354
16: myshopify.com
17: mashable.com
7.850628137588501
18: pixabay.com
4.517936706542969
19: dx.doi.org
4.596771478652954
20: www.plesk.com
4.414745807647705
21: www.zendesk.com
4.968740940093994
22: www.bmj.com
5.114487886428833
23: wpengine.com
5.908820867538452
24: www.pnas.org
4.363210916519165
25: www.patreon.com
4.548258304595947
26: www.thelancet.com
4.424659013748169
27: www.fao.org
4.871751070022583


4.803656101226807
214: wikitravel.org
4.524162769317627
215: www.acpjournals.org
4.320748805999756
216: www.rki.de
4.422112464904785
217: www.techdirt.com
4.66299843788147
218: rsf.org
4.653609752655029
219: socialmediaexaminer.com
5.370415210723877
220: www.visualcapitalist.com
4.56213116645813
221: ucsusa.org
5.187598943710327
222: www.udacity.com
4.448856830596924
223: unctad.org
5.705899715423584
224: www.rawstory.com
4.933297872543335
225: www.minds.com
4.309435844421387
226: www.nolo.com
4.289064168930054
227: www.annualreviews.org
4.439908742904663
228: thediplomat.com
5.078019380569458
229: www.mindtools.com
4.572747468948364
230: www.broadwayworld.com
4.777256727218628
231: petapixel.com
4.781234264373779
232: arabnews.com
4.999016523361206
233: www.commonsensemedia.org
4.798163652420044
234: doaj.org
4.397528409957886
235: www.lung.org
4.466034173965454
236: www.catchtiger.com
4.385030508041382
237: www.opendemocracy.net
4.676399230957031
238: www.sphinx-doc.org
4.28542661666

4.91300106048584
423: www.grandviewresearch.com
4.455998659133911
424: www.artofmanliness.com
4.393996477127075
425: besthookupwebsites.org
4.2739996910095215
426: avvo.com
6.092005491256714
427: www.creatorbyzmags.com
4.295996904373169
428: custom-roms.com
4.31599760055542
429: squareblogs.net
4.283010244369507
430: www.highsnobiety.com
4.485002279281616
431: clutch.co
6.447000026702881
432: www.iherb.com
4.3759989738464355
433: www.buymeacoffee.com
4.518998861312866
434: imgflip.com
4.324001789093018
435: www.nestle.com
5.562005043029785
436: beforeitsnews.com
4.543001413345337
437: personalbadcreditloans.net
4.280000448226929
438: www.amnestyusa.org
4.6659722328186035
439: www.jewishvirtuallibrary.org
4.5030012130737305
440: teamspeak.com
4.651993036270142
441: gizmodo.com.au
5.15500020980835
442: ahram.org.eg
4.436998128890991
443: www.iconfinder.com
4.512025833129883
444: voxeu.org
4.465974807739258
445: www.ziprecruiter.com
4.53299880027771
446: itsmyurls.com
4.318001985549927
44

4.5459980964660645
626: 00-tv.com
4.314002275466919
627: brightbrides.org
4.302996873855591
628: www.animenewsnetwork.com
4.468002557754517
629: wise.com
4.9730002880096436
630: happyrxpharmacy21.com
4.278000354766846
631: www.ilfattoquotidiano.it
4.339214086532593
632: redstate.com
4.608001708984375
633: www.cmswire.com
4.4530017375946045
634: titleloansusa.info
4.286999225616455
635: www.hubspot.com
4.583016872406006
636: xuypharmacyonline.com
4.283000230789185
637: www.procon.org
4.511993408203125
638: nationaltitleloan.net
4.282003402709961
639: oboporn.com
4.294999599456787
640: www.rockhall.com
4.540000677108765
641: www.argentina.gob.ar
4.403002738952637
642: admin.qsl.net
4.43000054359436
643: tinyblogging.com
4.2760009765625
644: fee.org
5.132999658584595
645: www.escapistmagazine.com
4.3950018882751465
646: www.sandiegoreader.com
4.781999588012695
647: www.nationalrail.co.uk
4.389999151229858
648: gleb.website
4.565002918243408
649: sildenapik.us
4.283000707626343
650: www.on

4.488994598388672
829: adult-cams.org
4.394000768661499
830: mb.com.ph
4.456003427505493
831: www.maxmind.com
4.428993225097656
832: www.joomlart.com
4.551000356674194
833: www.worldvision.org
4.725999355316162
834: www.phoronix.com
4.7209742069244385
835: www.joinclubhouse.com
4.419003248214722
836: paydayloansohio.org
4.284997463226318
837: www.forexfactory.com
4.351003646850586
838: yoursite.com
4.509963035583496
839: www.ancient-origins.net
4.608009099960327
840: www.sika.com
4.31996750831604
841: datingavis.fr
4.292999267578125
842: untappd.com
6.708997488021851
843: vwo.com
4.721000909805298
844: www.colby.edu
4.284622669219971
845: www.nairaland.com
4.318779468536377
846: www.miamidade.gov
4.449999809265137
847: insideclimatenews.org
4.80099892616272
848: www.xn--42c9bsq2d4f7a2a.com
4.370998382568359
849: caaliss.us
4.274003267288208
850: essaysrescue.com
4.3089985847473145
851: taylorandfrancis.com
5.056999206542969
852: paydayloansmichigan.org
4.277994155883789
853: www.sott.n

...

4.32527494430542
5214: www.rff.org
4.437453269958496
5215: paperwritingessayservice100.com
4.287173748016357
5216: essaywritingeie.com
4.2994935512542725
5217: www.javascript.com
4.397732734680176
5218: saveyoursite.date
4.2917869091033936
5219: orionmagazine.org
4.53884744644165
5220: www.veracode.com
4.491127014160156
5221: essayio.com
4.289030313491821
5222: 1studyhelp.com
4.3356709480285645
5223: www.apsanet.org
4.696828126907349
5224: agami360.com
4.36439323425293
5225: haicuneo.com
4.296949863433838
5226: essaysm.com
4.282522678375244
5227: creditscoresws.com
4.289717197418213
5228: www.sitecore.com
4.444387912750244
5229: jeyessay.com
4.288567543029785
5230: ipaperwriting.com
4.294174432754517
5231: essayswriting.org
4.60131311416626
5232: frewwebs.com
4.28435206413269
5233: bibletopicindex.com
4.302908182144165
5234: www.thedailymash.co.uk
4.704034090042114
5235: www.ismp.org
4.591127634048462
5236: www.amoeba.com
4.35439395904541
5237: www.38north.org
4.5622522830963135
5238: 

4.298283576965332
5412: opameds.com
4.290852069854736
5413: jweekly.com
4.579786062240601
5414: panampost.com
4.544856309890747
5415: www.plugshare.com
4.3442418575286865
5416: 1mail-order-brides.com
4.286632061004639
5417: metromsk.com
4.330873727798462
5418: www.thecitizen.co.tz
4.3037590980529785
5419: cryptolisting.org
4.325475692749023
5420: www.oekotest.de
4.5036301612854
5421: ivermectinep.com
4.305837869644165
5422: www.shopperapproved.com
4.522094249725342
5423: www.lemoniteur.fr
4.447845935821533
5424: strangesounds.org
4.74013614654541
5425: russkiymir.ru
5.129041433334351
5426: harz-flirt.com
4.287150144577026
5427: edussons.com
4.400811433792114
5428: islamqa.info
5.455248117446899
5429: bestpillsonly.com
4.284970283508301
5430: hentaiplus.co
4.690642356872559
5431: apicialis.com
4.293548822402954
5432: juiceplus.com
5.020178318023682
5433: rare.us
4.637480974197388
5434: wallhaven.cc
4.731442213058472
5435: greekmythology.com
4.685572624206543
5436: shopelvis.com
4.324257

4.289339065551758
5610: viagragenn.com
4.285794734954834
5611: www.petside.com
4.358227252960205
5612: www.icehotel.com
4.347291946411133
5613: www.tmforum.org
4.758049011230469
5614: bestseocompany.info
4.491000175476074
5615: www.thefirearmblog.com
4.875590085983276
5616: controlc.com
4.328843593597412
5617: spray.se
4.44140625
5618: onlinexpharmacy.com
4.294739246368408
5619: cannabisoilhemp.com
4.279900312423706
5620: privatenude.org
4.30448579788208
5621: creditscoresids.com
4.298978567123413
5622: duaxeoto.net
4.296325922012329
5623: cbdoinlinexo.com
4.281224727630615
5624: www.u-s-history.com
4.498163461685181
5625: myhealthsrc.com
4.315618276596069
5626: okfnlabs.org
4.614781856536865
5627: www.nhnieuws.nl
4.320403814315796
5628: parkbench.com
4.96709132194519
5629: www.swiss-miss.com
4.4977126121521
5630: translifeline.org
4.917983055114746
5631: medicationcialis.com
4.2845542430877686
5632: payforpapers.net
4.580634355545044
5633: opusdei.org
4.5533223152160645
5634: brides-f

4.28394341468811
5808: apa.az
4.393541574478149
5809: www.lemonade.com
4.492051362991333
5810: signlending.com
4.286082983016968
5811: www.prusaprinters.org
4.597684860229492
5812: pharmacymain.com
4.323343515396118
5813: prozacflxt.com
4.280225038528442
5814: www.mastersportal.com
4.6028358936309814
5815: www.toxel.com
4.484498977661133
5816: quadlending.com
4.293996572494507
5817: unblocked2.me
4.357301950454712
5818: pharmacyfive.com
4.2881739139556885
5819: xppharmacy.com
4.29290509223938
5820: westegg.com
4.504901170730591
5821: www.thrustmaster.com
4.494902849197388
5822: defipulse.com
4.433299541473389
5823: www.thecable.ng
4.61766242980957
5824: viagralm.com
4.294830799102783
5825: eightpills.com
4.30998420715332
5826: intelpills.com
4.286649703979492
5827: www.travelportland.com
4.671735763549805
5828: propranololsale.com
4.290894508361816
5829: mgtow.wiki
4.299285411834717
5830: www.lsa-conso.fr
4.38143253326416
5831: lettocash.com
4.292867422103882
5832: edcanada24.com
4.283

4.304880857467651
6010: sportsmansguide.com
4.643483638763428
6011: www.christianheadlines.com
4.4627885818481445
6012: ringsurf.com
4.680463552474976
6013: cialisplusviagra.com
4.291571855545044
6014: cheaphydroxychloroquine.com
4.29120945930481
6015: www.lme.com
4.9121692180633545
6016: edgenrx.com
4.302258014678955
6017: sildenafilforsale.com
4.287039041519165


In [26]:
#Output data into .csv file
#Load previous collection
name = "cf2"
file_name = name + '.csv'

df = pd.json_normalize(certList)
df.to_csv(file_name, index=False, header=True)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
df

,id,tbs_sha256,dns_names,pubkey_sha256,not_before,not_after,url,issuer.name,issuer.pubkey_sha256,issuer.operator.id,...,issuer. O,issuer. CN,cert.type,cert.sha256,cert.data,subject.CN,subject.O,subject.L,subject.ST,subject.C
0,2380262164,7a71b92da46190b0ee5851d959df3903f120a5799658b1...,[medium.com],97293eb373046958ed172274a09205e8971dd2823cfba3...,2021-03-18T00:00:00Z,2022-03-17T23:59:59Z,medium.com,"C=US, O=""Cloudflare, Inc."", CN=Cloudflare Inc ...",144cd5394a78745de02346553d126115b48955747eb909...,17,...,"""Cloudflare Inc.""",Cloudflare Inc ECC CA-3,precert,663636347f63b5647843276ede9e549e3fd204de194af4...,MIIDlzCCAzygAwIBAgIQC75g6MjTzJu6vRTO5LMTEjAKBg...,medium.com,Cloudflare Inc.,San Francisco,California,US
1,2675470370,4ccfa6925ac0c9bc1a533fcc8e317d10be6e84614f5de6...,"[*.creativecommons.org, creativecommons.org, s...",a2eead2dceb2f6dc37c687b2e20395d10a284c09440c2c...,2021-06-29T00:00:00Z,2022-06-28T23:59:59Z,creativecommons.org,"C=US, O=""Cloudflare, Inc."", CN=Cloudflare Inc ...",144cd5394a78745de02346553d126115b48955747eb909...,17,...,"""Cloudflare Inc.""",Cloudflare Inc ECC CA-3,precert,6709bf0ec86e5c6664ecceb64f76637ebb466aacf592d6...,MIID2DCCA36gAwIBAgIQCsnB1dqUteTBh8btdJ73vzAKBg...,sni.cloudflaressl.com,Cloudflare Inc.,San Francisco,California,US
2,2731582425,319d3cb9c985783dab7df83e9ab757f15d59b8381ba02f...,"[sni.cloudflaressl.com, www.who.int]",0086e3c4f2d1192645a8ce30f5e9e85edb21e40b0493b4...,2021-07-17T00:00:00Z,2022-07-16T23:59:59Z,www.who.int,"C=US, O=""Cloudflare, Inc."", CN=Cloudflare Inc ...",144cd5394a78745de02346553d126115b48955747eb909...,17,...,"""Cloudflare Inc.""",Cloudflare Inc ECC CA-3,precert,fe32eb093ec8fecb01e0b550d70d79de6c576422e489fa...,MIIDuDCCA1+gAwIBAgIQDznLvIxBx97cfhpbL8F4kTAKBg...,sni.cloudflaressl.com,Cloudflare Inc.,San Francisco,California,US
3,2687186104,5180e273a5506fc33b2ee8defc9f0764f477e4e71d3b96...,"[sni.cloudflaressl.com, *.tinyurl.com, tinyurl...",db6f32e5502553bdcf115477353db0a25ff38fa18b4803...,2021-07-03T00:00:00Z,2022-07-02T23:59:59Z,tinyurl.com,"C=US, O=""Cloudflare, Inc."", CN=Cloudflare Inc ...",144cd5394a78745de02346553d126115b48955747eb909...,17,...,"""Cloudflare Inc.""",Cloudflare Inc ECC CA-3,precert,9730a82f1c9f64c109a79d38870bca044295e7754bffd2...,MIIDyTCCA26gAwIBAgIQDicoIx5WrdUGpyJnH/BscTAKBg...,sni.cloudflaressl.com,Cloudflare Inc.,San Francisco,California,US
4,2727854348,d41e4acd8ff48d217cd66fd2eab961c65d91d0f3ea0915...,"[*.cloudflare.com, cloudflare.com, sni.cloudfl...",b69f61bd0c22d5d4738ed5a86caafdbebc9ff2ddcfdbca...,2021-07-16T00:00:00Z,2022-07-15T23:59:59Z,www.cloudflare.com,"C=US, O=""Cloudflare, Inc."", CN=Cloudflare Inc ...",144cd5394a78745de02346553d126115b48955747eb909...,17,...,"""Cloudflare Inc.""",Cloudflare Inc ECC CA-3,precert,edbafff04b0a585f232c8f19c024651be82ae0cd65ae8e...,MIIDzTCCA3SgAwIBAgIQBYVZigOtZWUZveAN25knGzAKBg...,sni.cloudflaressl.com,Cloudflare Inc.,San Francisco,California,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5996,2222293771,225acb96ae75dba26f0732921f6765fc4059081381b099...,"[*.cialisplusviagra.com, cialisplusviagra.com,...",5dbe5cbdde0f48c3e7c508bfc057a6b10fb07d80ba9ebf...,2021-01-16T00:00:00Z,2022-01-15T23:59:59Z,cialisplusviagra.com,"C=US, O=""Cloudflare, Inc."", CN=Cloudflare Inc ...",144cd5394a78745de02346553d126115b48955747eb909...,17,...,"""Cloudflare Inc.""",Cloudflare Inc ECC CA-3,precert,89d7c336bbe7be78689fef5da66ac2ba4f8cf4b85bf20a...,MIID0zCCA3igAwIBAgIQCw8CVjLFw7K1foBKpJX91zAKBg...,sni.cloudflaressl.com,Cloudflare Inc.,San Francisco,CA,US
5997,2486323189,f9ad3cab57166a67dca87710344c96532bfdaf127b9c07...,"[*.cheaphydroxychloroquine.com, cheaphydroxych...",d9e329af0fa70bce8406050f55b0f791b17272426acd36...,2021-04-26T00:00:00Z,2022-04-25T23:59:59Z,cheaphydroxychloroquine.com,"C=US, O=""Cloudflare, Inc."", CN=Cloudflare Inc ...",144cd5394a78745de02346553d126115b48955747eb909...,17,...,"""Cloudflare Inc.""",Cloudflare Inc ECC CA-3,precert,bd1d92c35945e1f1c4347097f7b2b

In [ ]:
statistics.mean(timeperiod)

In [13]:
cagroupsf = df['issuer.operator.name'].value_counts()
cagroupsf

DigiCert                                                1366
Internet Security Research Group                        1092
Sectigo                                                  536
GoDaddy                                                   67
GlobalSign nv-sa                                          25
Entrust                                                   13
Google Trust Services LLC                                  5
SecureTrust                                                4
QuoVadis                                                   3
Asseco Data Systems S.A. (previously Unizeto Certum)       1
Buypass                                                    1
SwissSign AG                                               1
SSL.com                                                    1
Name: issuer.operator.name, dtype: int64